In [1]:
import pandas as pd
import numpy as np

In [2]:
FILE = 'travistorrent_11_1_2017.csv'
size = 500000
#size = 2000000

In [3]:
#for dataset in pd.read_csv(file, engine='c', chunksize=size):
#    process(dataset)

In [4]:
%%time
dataset = pd.read_csv(FILE, sep=',', engine='c',dtype={'gh_team_size': np.int32,
                                                        'git_num_all_built_commits': np.int32,
                                                        'gh_num_commit_comments': np.int32,
                                                        'git_diff_src_churn': np.int32,
                                                        'git_diff_test_churn': np.int32,
                                                        'gh_diff_files_added': np.int32,
                                                        'gh_diff_files_deleted': np.int32,
                                                        'gh_diff_files_modified': np.int32,
                                                        'gh_diff_tests_added': np.int32,
                                                        'gh_diff_tests_deleted': np.int32,
                                                        'gh_diff_src_files': np.int32,
                                                        'gh_diff_doc_files': np.int32,
                                                        'gh_diff_other_files': np.int32,
                                                        'gh_num_commits_on_files_touched': np.int32,
                                                        'gh_sloc': np.int32,
                                                        'tr_build_number': np.int32,
                                                        'tr_job_id': np.int32,
                                                        'tr_build_id': np.float32,
                                                        'gh_pull_req_num': np.float32,
                                                        'gh_num_commits_in_push': np.float32,
                                                        'tr_prev_build': np.float32,
                                                        'gh_num_issue_comments': np.float32,
                                                        'gh_num_pr_comments': np.float32,
                                                        'gh_test_lines_per_kloc': np.float32,
                                                        'gh_test_cases_per_kloc': np.float32,
                                                        'gh_asserts_cases_per_kloc': np.float32,
                                                        'gh_description_complexity': np.float32,
                                                        'tr_duration': np.float32,
                                                        'tr_log_setup_time': np.float32,
                                                        'tr_log_num_tests_ok': np.float32,
                                                        'tr_log_num_tests_failed': np.float32,
                                                        'tr_log_num_tests_run': np.float32,
                                                        'tr_log_num_tests_skipped': np.float32,
                                                        'tr_log_testduration': np.float32,
                                                        'tr_log_buildduration': np.float32,

                                                        'gh_project_name': 'category',
                                                        'gh_pr_created_at': 'category',
                                                        'gh_lang': 'category',
                                                        'git_merged_with': 'category',
                                                        'git_branch': 'category',
                                                        'gh_commits_in_push': 'category',
                                                        'git_prev_commit_resolution_status': 'category',
                                                        'git_prev_built_commit': 'category',
                                                        'gh_first_commit_created_at': 'category',
                                                        'git_all_built_commits': 'category',
                                                        'git_trigger_commit': 'category',
                                                        'tr_virtual_merged_into': 'category',
                                                        'tr_original_commit': 'category',
                                                        'gh_pushed_at': 'category',
                                                        'gh_build_started_at': 'category',
                                                        'tr_status': 'category',
                                                        'tr_jobs': 'category',
                                                        'tr_log_lan': 'category',
                                                        'tr_log_status': 'category',
                                                        'tr_log_analyzer': 'category',
                                                        'tr_log_frameworks': 'category',
                                                        'tr_log_bool_tests_failed': 'category',
                                                        'tr_log_tests_failed': 'category',


                                                        'gh_is_pr': np.bool,
                                                        'gh_by_core_team_member': np.bool,
                                                        'tr_log_bool_tests_ran': np.bool}

                                                                                          , nrows = size, encoding='utf-8')
# nrows=1000000
# dtype={'tr_build_id': np.int32}
# ,dtype={'gh_num_commits_in_push': np.float32}
'''
Signature:
    sep - separa por virgula
    nrows - limite de linhas lidas
    encoding - tipo de caracter
    engine - Mecanismo do analisador
    na_values - procura os padrões na|nan como str
    keep_default_na - define como nan os padroes econtrados em na_values
    
'''



CPU times: user 7.28 s, sys: 268 ms, total: 7.55 s
Wall time: 7.55 s


In [4]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [5]:
props, NAlist = reduce_mem_usage(pd.read_csv(FILE))

/home/wagner/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,6,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Memory usage of properties dataframe is : 1649.0096521377563  MB
******************************
Column:  tr_build_id
dtype before:  int64
dtype after:  uint32
******************************
******************************
Column:  gh_is_pr
dtype before:  bool
dtype after:  uint8
******************************
******************************
Column:  gh_pull_req_num
dtype before:  float64
dtype after:  uint16
******************************
******************************
Column:  gh_num_commits_in_push
dtype before:  float64
dtype after:  uint8
******************************
******************************
Column:  tr_prev_build
dtype before:  float64
dtype after:  uint32
******************************
******************************
Column:  gh_team_size
dtype before:  int64
dtype after:  uint16
******************************
******************************
Column:  git_num_all_built_commits
dtype before:  int64
dtype after:  uint16
******************************
****************************

In [8]:
NAlist

['gh_pull_req_num',
 'gh_num_commits_in_push',
 'tr_prev_build',
 'gh_num_issue_comments',
 'gh_num_pr_comments',
 'gh_description_complexity',
 'tr_duration',
 'tr_log_setup_time',
 'tr_log_num_tests_ok',
 'tr_log_num_tests_failed',
 'tr_log_num_tests_run',
 'tr_log_num_tests_skipped',
 'tr_log_testduration',
 'tr_log_buildduration']

In [8]:
dataset['git_merged_with'].nbytes

4000000

In [9]:
# dataset.info()

In [10]:
dataset['git_merged_with'] = dataset['git_merged_with'].astype('category')


In [11]:
dataset['git_merged_with'].nbytes

500024

In [20]:
4000000
500024


0                  NaN
1                  NaN
2                  NaN
3                  NaN
4                  NaN
5                  NaN
6                  NaN
7                  NaN
8                  NaN
9                  NaN
10                 NaN
11                 NaN
12                 NaN
13                 NaN
14                 NaN
15                 NaN
16                 NaN
17                 NaN
18                 NaN
19                 NaN
20                 NaN
21                 NaN
22                 NaN
23                 NaN
24                 NaN
25                 NaN
26                 NaN
27                 NaN
28                 NaN
29                 NaN
              ...     
499970    merge_button
499971    merge_button
499972    merge_button
499973             NaN
499974             NaN
499975             NaN
499976             NaN
499977             NaN
499978             NaN
499979             NaN
499980             NaN
499981             NaN
499982     

<font size=4/>
   * Elimia as colunas não utilizadas

In [10]:
# Eliminando colunas
props.drop([  'gh_diff_files_added',
                'gh_commits_in_push',
                'gh_diff_files_deleted',
                'gh_diff_files_modified',
                'gh_diff_tests_added',
                'gh_diff_tests_deleted',
                'gh_diff_src_files',
                'gh_diff_doc_files',
                'gh_diff_other_files',
                'gh_sloc',
                'gh_test_lines_per_kloc',
                'gh_test_cases_per_kloc',
                
                'tr_build_number',
                'tr_duration',
                'tr_log_lan',
                'tr_log_num_tests_run',
                'tr_log_setup_time',
                'tr_log_analyzer',
                'tr_log_frameworks',
                'tr_log_bool_tests_ran',
                'tr_log_bool_tests_failed',
                'tr_log_tests_failed',
                'tr_log_testduration',
                'tr_log_buildduration'], axis=1, inplace=True)

In [14]:
df_ruby = props.loc[props['gh_project_name']=='rails/rails']

df_ruby = df_ruby.dropna() # Remove as linhas NAN

In [15]:
df_ruby

,tr_build_id,gh_project_name,gh_is_pr,gh_pr_created_at,gh_pull_req_num,gh_lang,git_merged_with,git_branch,gh_num_commits_in_push,git_prev_commit_resolution_status,...,gh_description_complexity,gh_pushed_at,gh_build_started_at,tr_status,tr_jobs,tr_job_id,tr_log_status,tr_log_num_tests_ok,tr_log_num_tests_failed,tr_log_num_tests_skipped
638680,8113937,rails/rails,1,2013-06-12 10:21:13,10925,ruby,merge_button,master,2,build_found,...,82,2013-02-17 01:06:32,2013-06-15 12:47:57,passed,"[8113949, 8113938, 8113948, 8113947, 8113944, ...",8113939,ok,14749,0,2
638681,8113937,rails/rails,1,2013-06-12 10:21:13,10925,ruby,merge_button,master,2,build_found,...,82,2013-02-17 01:06:32,2013-06-15 12:47:57,passed,"[8113949, 8113938, 8113948, 8113947, 8113944, ...",8113941,ok,7260,0,44
638682,8113937,rails/rails,1,2013-06-12 10:21:13,10925,ruby,merge_button,master,2,build_found,...,82,2013-02-17 01:06:32,2013-06-15 12:47:57,passed,"[8113949, 8113938, 8113948, 8113947, 8113944, ...",8113943,ok,7630,0,77
638683,8113937,rails/rails,1,2013-06-12 10:21:13,10925,ruby,merge_button,master,2,build_found,...,82,2013-06-15 12:45:17,2013-06-15 12:47:57,passed,"[8113949, 8113938, 8113948, 8113947, 8113944, ...",8113944,ok,879,0,0
638684,8113937,rails/rails,1,2013-06-12 10:21:13,10925,ruby,merge_button,master,2,build_found,...,82,2013-06-15 12:45:17,2013-06-15 12:47:57,passed,"[8113949, 8113938, 8113948, 8113947, 8113944, ...",8113949,ok,7464,0,96
638685,8113937,rails/rails,1,2013-06-12 10:21:13,10925,ruby,merge_button,master,2,build_found,...,82,2013-02-17 01:12:04,2013-06-15 12:47:57,passed,"[8113949, 8113938, 8113948, 8113947, 8113944, ...",8113946,ok,3524,0,12
638686,8113937,rails/rails,1,2013-06-12 10:21:13,10925,ruby,merge_button,master,2,build_found,...,82,2013-02-17 01:12:04,2013-06-15 12:47:57,passed,"[8113949, 8113938, 8113948, 8113947, 8113944, ...",8113940,ok,7181,0,23
638687,8113937,rails/rails,1,2013-06-12 10:21:13,10925,ruby,merge_button,master,2,build_found,...,82,2013-02-17 01:17:33,2013-06-15 12:47:57,passed,"[8113949, 8113938, 8113948, 8113947, 8113944, ...",8113947,ok,7336,0,46
638688,8113937,rails/rails,1,2013-06-12 10:21:13,10925,ruby,merge_button,master,2,build_found,...,82,2013-02-17 01:49:43,2013-06-15 12:47:57,passed,"[8113949, 8113938, 8113948, 8113947, 8113944, ...",8113942,ok,7115,0,57
638689,8113937,rails/rails,1,2013-06-12 10:21:13,10925,ruby,merge_button,master,2,build_found,...,82,2013-02-17 01:52:24,2013-06-15 12:47:57,passed,"[8113949, 8113938, 8113948, 8113947, 8113944, ...",8113938,ok,851,0,0


<font size=4/> 
   * Filtra o projeto, remove linhas NAN, separa a coluna gh_pushed_at

In [8]:
df_ruby = dataset.loc[dataset['gh_project_name']=='rails/rails'] # Filtro penas o projeto rails/rails

In [9]:
df_ruby = df_ruby.dropna() # Remove as linhas NAN

In [10]:
df_ruby['data'], df_ruby['hora'] = df_ruby.gh_pushed_at.str.split(' ').str # Separa a coluna que possui data e hora em duas novas colunas de data hora

<font size=4/> 
   * Remove linhas duplicadas

In [11]:
single_push = df_ruby.drop_duplicates(['tr_build_id']) # remove as linhas duplicadas

single_push = single_push['tr_build_id'] # cria uma serie com os push unicos do projeto


<font size=4/> 
   * Soma de todos push

In [12]:
num_commit = df_ruby.gh_num_commits_in_push.sum() 

<font size=4/> 
   * Duração do projeto

In [13]:
first_data, hora = df_ruby.gh_first_commit_created_at.head(1).str.split(' ').str
first_data = int(first_data.str.replace('-', ''))

In [14]:
last_data,  hora = df_ruby.gh_first_commit_created_at.tail(1).str.split(' ').str
last_data = int(last_data.str.replace('-',''))

In [15]:
data_commit = last_data - first_data

<font size=4/> 
   * Media participante

In [16]:
media_participante = df_ruby.gh_team_size.sum()/ len(single_push)

<font size=4/> 
   * teste

In [17]:
taxa_commit = num_commit/(data_commit * media_participante)

In [18]:
taxa_commit

0.0008368904169903687